In [1412]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import sparse
from scipy import optimize
from scipy import signal
from functools import reduce
from collections import defaultdict
import pprint
from tqdm.notebook import tqdm

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [709]:
def get_num_baselines(Nside):
    # For square of N telescopes, should be 2N-2*sqrt(N) + 1
    # Fix to a corner and there are N-1 options for unique baselines. 
    # Flip over to other side (over x or y axis) and get another N-1 options
    # Duplicated are the pure x-axis and pure y-axis so -2*(sqrt(N)-1)
    # Final +1 is from 0 baseline
    
    N_bases = 2*Nside**2 - 2*Nside
    return int(N_bases)

rand_phases = lambda x: np.random.uniform(0, 2*np.pi, x)
zero_weight = lambda x, d: x/d if d else 0

def make_gains(Nside):
    # Create complex gains with either (amplitude, phase) or (real, imaginary)
    Nant = Nside**2
    gain_amp = np.random.normal(1, .05, Nant)
    gain_phase = rand_phases(Nant)
    tgain = gain_amp*np.exp(1j*gain_phase)    
    return tgain

def make_data(Nside, gains, noise=0.1):
    Nant = Nside**2
    Nbase = get_num_baselines(Nside)
    vis_true = np.random.normal(0,1,size=(Nbase,2)).view(np.complex128).flatten() ## size of unique baselines
    ant_i, ant_j, visndx, data = [], [], [], []
    ndx=0
    ndx2base={}
    base2ndx={}
    for i in range(Nant):
        xi,yi=np.unravel_index(i,(Nside,Nside))
        for j in range (i+1,Nant):
            xj,yj=np.unravel_index(j,(Nside,Nside))
            assert (xj>=xi)
            baseline = (xj-xi,yj-yi)
            if baseline in base2ndx:
                cndx = base2ndx[baseline]
            else:
                cndx = ndx
                base2ndx[baseline]=ndx
                ndx2base[ndx]=baseline
                ndx+=1
            ant_i.append(i)
            ant_j.append(j)
            visndx.append(cndx)
            data.append(vis_true[cndx]*gains[i]*np.conj(gains[j]))
            
    assert(ndx==Nbase)
    ant_i = np.array(ant_i)
    ant_j = np.array(ant_j)
    visndx = np.array(visndx)
    data = np.array(data)
    noise = np.random.normal(0,noise,size=(len(data),2)).view(np.complex128).flatten() ## size of unique baselines
    data += noise
    return vis_true, data, ant_i, ant_j, visndx, ndx2base, base2ndx

In [728]:
def make_uv_grid(Nside):
    uv_size = Nside*2 - 1
    center = (Nside-1,Nside-1)
    npcenter = np.array(center)
    random_uv_grid = np.zeros((uv_size, uv_size, 2)).view(np.complex128)
    poss_uv = random_uv_grid.reshape((uv_size, uv_size))
    
    # Bottom right and top left should be conjugates
    for i in range(Nside-1, uv_size):
        for j in range(Nside, uv_size):
            rel_to_center = np.array([i,j]) - npcenter
            conj_pos = tuple(npcenter - rel_to_center)
            visb = np.random.normal(0,1,2).view(np.complex128)
            poss_uv[i,j] = visb
            poss_uv[conj_pos] = np.conj(visb)
    
    # Top right and bottom left should be conjugates
    for i in range(Nside, uv_size):
        for j in range(0, Nside):
            rel_to_center = np.array([i,j]) - npcenter
            conj_pos = tuple(npcenter - rel_to_center)
            visb = np.random.normal(0,1,2).view(np.complex128)
            poss_uv[i,j] = visb
            poss_uv[conj_pos] = np.conj(visb)
            
    poss_uv[center] = 0
    return poss_uv, npcenter

In [1961]:
def make_data_grid(Nside, gains, weight_beam, noise=0.1):
    Nant = Nside**2
    Nbase = get_num_baselines(Nside)
    beam1_f = weight_beam
    beamsize = weight_beam.shape[0]
    beam_radius = int((beamsize-1)/2)
    uv_grid, npcenter = make_uv_grid(Nside+beam_radius)
    vis_true = np.zeros(Nbase, dtype=np.complex128)
    ant_i, ant_j, visndx, data, data_indx = [], [], [], [], []
    ndx=0
    ndx2base={}
    base2ndx={}
    for i in range(Nant):
        xi,yi=np.unravel_index(i,(Nside,Nside))
        for j in range (i+1,Nant):
            xj,yj=np.unravel_index(j,(Nside,Nside))
            assert (xj>=xi)
            baseline = (xj-xi,yj-yi)
            if baseline in base2ndx:
                cndx = base2ndx[baseline]
            else:
                cndx = ndx
                base2ndx[baseline]=ndx
                ndx2base[ndx]=baseline
                ndx+=1
            center = npcenter + baseline
            uv_indices = tuple(np.meshgrid(range(center[0]-beam_radius,center[0]+beam_radius+1),range(center[1]-beam_radius,center[1]+beam_radius+1)))
            uv_points = uv_grid[uv_indices]*weight_beam
            vis_true[cndx] = uv_grid[tuple(center)]
            ant_i.append(i)
            ant_j.append(j)
            visndx.append(cndx)
            data.append(np.sum(signal.fftconvolve(uv_points, beam1_f).T*gains[i]*np.conj(gains[j])))
            data_indx.append(uv_indices)
            
    assert(ndx==Nbase)
    ant_i = np.array(ant_i)
    ant_j = np.array(ant_j)
    visndx = np.array(visndx)
    data = np.array(data)
    noise = np.random.normal(0,noise,size=(len(data),2)).view(np.complex128).flatten() ## size of unique baselines
    data += noise
    return vis_true, data, ant_i, ant_j, visndx, data_indx, uv_grid, ndx2base, base2ndx

In [1892]:
def make_pred(gains, vis, ant_i, ant_j, visndx):
    gains_i = gains[ant_i]
    cgains_j = np.conj(gains[ant_j])
    pred = gains_i*cgains_j*vis[visndx]
    return pred

In [1893]:
def chi2 (data, gains, vis, ant_i, ant_j, visndx, noise=0.1):
    pred = make_pred(gains, vis, ant_i, ant_j, visndx)
    chi2 = np.abs((data - pred)**2).sum()/(noise**2)
    dof = len(data)*2
    return chi2, dof

In [1894]:
def flat_to_matrix(Nside, flattened, a_i, a_j):
    Nant = Nside**2
    temp_mat = np.zeros((Nant, Nant), dtype=np.complex128)
    temp_mat[a_i, a_j] = flattened
    temp_mat[a_j, a_i] = np.conj(flattened)
    return temp_mat

In [1953]:
def solve_grid(data, data_ndx, scale=1, beamweight=None):
   
    new_uv_size = max(list(map(np.max, data_ndx)))+1
    new_uv_indx = np.arange(new_uv_size**2).reshape((new_uv_size, new_uv_size))
    real2ndx = {}
    ndx2real = {}
    real_counter = 0
    if beamweight is None:
        beamweight = np.ones(data_ndx[0][0].flatten().shape[0])
    
    for i in data_ndx:
        for k in set(new_uv_indx[i].flatten()):
            if k not in real2ndx:
                real2ndx[k] = real_counter
                ndx2real[real_counter] = k
                real_counter += 1
    vector_real = np.vectorize(real2ndx.get)
    vector_ndx = np.vectorize(ndx2real.get)
    
    A = np.zeros((data_len, len(ndx2real)))
    b = data/scale
    for i,v in enumerate(data_ndx):
        sky_ndxs = new_uv_indx[v].flatten()
        non_zeros = vector_real(sky_ndxs)
        A[i][non_zeros] = beamweight
    shitsol = np.linalg.lstsq(A,b, rcond=None)[0]
    score = np.sum(np.abs(A@shitsol - b))
    return shitsol, ndx2real, real2ndx, score, A, b

In [1896]:
def get_weighted_array(alpha, Nspacing, numdraws=1e5):

    Nbeam = Nspacing**2
    rmax = alpha*.5
    ndraws = int(numdraws)
    spacing = np.linspace(0,1,Nspacing+1)
    centered_spacing = spacing - .5
    empty_weight_beam = np.zeros((Nspacing, Nspacing))
    
    for i in range(Nbeam):
        xi, yi = np.unravel_index(i, (Nspacing, Nspacing))
        draws = np.array([np.random.uniform(centered_spacing[xi], centered_spacing[xi+1], ndraws), np.random.uniform(centered_spacing[yi], centered_spacing[yi+1], ndraws)])
        dist = np.linalg.norm(draws, axis=0)
        empty_weight_beam[xi, yi] = np.sum(dist < rmax)/num_draws
    sym_beam = .5*(empty_weight_beam + empty_weight_beam.T)
    return sym_beam

In [1980]:
Nside = 5
Nant = Nside * Nside
Nbase = get_num_baselines(Nside)
# gains_true = make_gains(Nside)
gains_true = np.ones(Nant, dtype=np.complex128)
# uv_grid, npcenter = make_uv_grid(Nside)
# vis_true, data, ant_i, ant_j, visndx, ndx2base, base2ndx = make_data(Nside, gains_true)
weighted_beam = get_weighted_array(.8, 3)
vis_true, data, ant_i, ant_j, visndx, data_ndx, uv_grid, ndx2base, base2ndx = make_data_grid(Nside, gains_true, weighted_beam, noise=0.1)
data_len = len(data)
no_noise = make_pred(gains_true, vis_true, ant_i, ant_j, visndx)

In [1981]:
# print(np.sum(signal.convolve(weighted_beam, weighted_beam)))
# print(np.sum(weighted_beam))
# print(np.sum(uv_grid[data_ndx[0]]))
print(data[0]/np.sum(uv_grid[data_ndx[0]]))
# print(data[0]/np.sum(weighted_beam))

(6.8020242745627275-2.7010526117826834j)


In [1982]:
sol, n2r, _, score, A, b = solve_grid(data, data_ndx, np.sum(weighted_beam), weighted_beam.flatten())
finsize = np.product(uv_grid.shape)
real_sol = np.array([uv_grid[np.unravel_index(n2r[i], uv_grid.shape)] for i in np.arange(len(n2r))])

In [1986]:
np.sum(np.abs(A@real_sol - b))

8.17824163971092

In [1925]:
phased_gains = gains_true*np.exp(1j*rand_phases(Nant)/5)
chi2(data, gains_true, vis_true, ant_i, ant_j, visndx)

(1918227.0581338815, 600)

### Below this is my random testing/playground

In [1342]:
uv_grid, npcenter = make_uv_grid(Nside+1)

In [1869]:
beam_size = 3
beam_radius = 1
center = tuple(npcenter+(3,0))

In [1870]:
beam1_f = np.ones((beam_size,beam_size))
beam2_f = np.zeros((9,9))
grab_3uv = np.zeros((3,3), dtype=np.complex128)

In [1871]:
conv_beam = signal.convolve(beam1_f, beam1_f)

In [1872]:
lhs = signal.convolve(beam1_f, uv_grid[center[0]-beam_radius:center[0]+beam_radius+1,center[1]-beam_radius:center[1]+beam_radius+1])

In [1873]:
center

(8, 5)

In [1874]:
uv_grid[7,5]

(-0.28916879703790555-0.00415327649319357j)

In [1876]:
lhs.real

array([[ 8.29324085e-01,  5.40155288e-01,  2.79812923e+00,
         1.96880514e+00,  2.25797394e+00],
       [ 2.01181620e+00,  7.88935884e-01,  3.54302605e+00,
         1.53120985e+00,  2.75409017e+00],
       [ 9.31314099e-01,  5.76049880e-01,  2.83118508e+00,
         1.89987098e+00,  2.25513520e+00],
       [ 1.01990014e-01,  3.58945919e-02,  3.30558512e-02,
        -6.89341623e-02, -2.83874067e-03],
       [-1.08050210e+00, -2.12886004e-01, -7.11840975e-01,
         3.68661126e-01, -4.98954971e-01]])

In [1858]:
np.sum(lhs)

(25.48066569885156-16.240879940093766j)